# Môn: Máy học nâng cao
* Sinh viên: Huỳnh Thị Bảo Trân
* MSSV: 19110482
* Bài thực hành: Lab05

In [1]:
!nvidia-smi

Wed Nov 23 08:53:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

--2022-11-23 08:53:58--  https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/plain]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.004s  

2022-11-23 08:53:59 (115 MB/s) - ‘spam.csv’ saved [503663/503663]



In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 31.2 MB/s 
     |████████████████████████████████| 182 kB 43.1 MB/s 
     |████████████████████████████████| 7.6 MB 46.3 MB/s 


In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModel, AutoConfig

In [5]:
data = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")

In [6]:
data.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis=1, inplace=True)

In [7]:
data 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [9]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data["gt"], test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [11]:
sentences=data['text']
labels=data['gt']
len(sentences),len(labels)

(5572, 5572)

In [12]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'bert-base-uncased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, add_special_tokens=True,
                                                max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
def Tokenizer(sentences):
    input_ids=[]
    attention_masks=[]

    for sent in tqdm(sentences):
        bert_inp=tokenizer.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

In [15]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 8343.49it/s]


In [16]:
def create_model(config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [17]:
model = create_model()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [19]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [20]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=64, 
                    epochs=10,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/10


53/53 [==============================] - 107s 2s/step - loss: 0.4561 - accuracy: 0.7579 - val_loss: 0.1633 - val_accuracy: 0.9623
Epoch 2/10
53/53 [==============================] - 91s 2s/step - loss: 0.2020 - accuracy: 0.9213 - val_loss: 0.0615 - val_accuracy: 0.9812
Epoch 3/10
53/53 [==============================] - 93s 2s/step - loss: 0.1244 - accuracy: 0.9509 - val_loss: 0.0512 - val_accuracy: 0.9848
Epoch 4/10
53/53 [==============================] - 94s 2s/step - loss: 0.1146 - accuracy: 0.9539 - val_loss: 0.0559 - val_accuracy: 0.9857
Epoch 5/10
53/53 [==============================] - 94s 2s/step - loss: 0.1139 - accuracy: 0.9545 - val_loss: 0.0473 - val_accuracy: 0.9883
Epoch 6/10
53/53 [==============================] - 94s 2s/step - loss: 0.0982 - accuracy: 0.9626 - val_loss: 0.0439 - val_accuracy: 0.9901
Epoch 7/10
53/53 [==============================] - 88s 2s/step - loss: 0.0807 - accuracy: 0.9755 - val_loss: 0.0667 - val_accuracy: 0.9865
Epoch 8/10
53/53 [============

In [21]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 10s 281ms/step - loss: 0.0269 - accuracy: 0.9928


[0.026902275159955025, 0.9928250908851624]

## Bài tập
1. Chạy mô hình bert-base-cased để xem kết quả thế nào
2. Thay Bert bằng xlnet-base-cased để chạy thử xem kết quả thế nào
3. Theo như bài tập số 2 ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10

# Bài làm

1. Chạy mô hình bert-base-cased để xem kết quả thế nào

In [22]:
data = pd.read_csv('/content/spam.csv', encoding = 'ISO-8859-1')

In [23]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [24]:
data 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [25]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [26]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [27]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['gt'], test_size = 0.2, random_state = 0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

In [28]:
sentences = data['text']
labels = data['gt']
len(sentences), len(labels)

(5572, 5572)

In [29]:
MAX_LENGTH = 128
NUM_CLASSES = len(data['gt'].unique())
MODEL_NAME = 'bert-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [30]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case = False, add_special_tokens = True, 
                                          max_length = MAX_LENGTH, pad_to_max_length = True, use_fast = True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [31]:
def Tokenizer(sentences):
    input_ids = []
    attention_masks = []

    for sent in tqdm(sentences):
        bert_inp = tokenizer.encode_plus(sent, add_special_tokens = True , max_length = MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    return input_ids, attention_masks

In [32]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 8691.93it/s]


In [33]:
def create_model(config = MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape = (MAX_LENGTH,), name = 'input_token', dtype = 'int32')
    input_masks_in = tf.keras.layers.Input(shape =(MAX_LENGTH,), name = 'masked_token', dtype = 'int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation = 'relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)
    model = tf.keras.Model(inputs = [input_ids_in, input_masks_in], outputs = X)
    return model

In [34]:
model = create_model()

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [36]:
model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-6), metrics = ['accuracy'])

In [37]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath = './best_model_1.hdf5', monitor = 'val_accuracy', verbos = 1, save_best_only = True, mode = 'max')
history = model.fit([input_ids_train, attention_masks_train], y_train, batch_size = 64, epochs = 10, callbacks = [mc], validation_data = ([input_ids_valid, attention_masks_vaild], y_valid))

Epoch 1/10


53/53 [==============================] - 110s 2s/step - loss: 0.3628 - accuracy: 0.8654 - val_loss: 0.1013 - val_accuracy: 0.9794
Epoch 2/10
53/53 [==============================] - 95s 2s/step - loss: 0.1040 - accuracy: 0.9593 - val_loss: 0.0830 - val_accuracy: 0.9839
Epoch 3/10
53/53 [==============================] - 95s 2s/step - loss: 0.0716 - accuracy: 0.9800 - val_loss: 0.0624 - val_accuracy: 0.9857
Epoch 4/10
53/53 [==============================] - 95s 2s/step - loss: 0.0579 - accuracy: 0.9823 - val_loss: 0.0642 - val_accuracy: 0.9865
Epoch 5/10
53/53 [==============================] - 89s 2s/step - loss: 0.0428 - accuracy: 0.9871 - val_loss: 0.0747 - val_accuracy: 0.9848
Epoch 6/10
53/53 [==============================] - 89s 2s/step - loss: 0.0354 - accuracy: 0.9895 - val_loss: 0.0689 - val_accuracy: 0.9857
Epoch 7/10
53/53 [==============================] - 89s 2s/step - loss: 0.0316 - accuracy: 0.9898 - val_loss: 0.0950 - val_accuracy: 0.9839
Epoch 8/10
53/53 [============

In [38]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 10s 282ms/step - loss: 0.0409 - accuracy: 0.9937


[0.0408785305917263, 0.9937219619750977]

2. Thay Bert bằng xlnet-base-cased để chạy thử xem kết quả thế nào

In [39]:
data = pd.read_csv('/content/spam.csv', encoding = 'ISO-8859-1')

In [40]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [41]:
data 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [42]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [43]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [44]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['gt'], test_size = 0.2, random_state = 0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

In [45]:
sentences = data['text']
labels = data['gt']
len(sentences), len(labels)

(5572, 5572)

In [46]:
MAX_LENGTH = 128
NUM_CLASSES = len(data['gt'].unique())
MODEL_NAME = 'xlnet-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [47]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case = False, add_special_tokens = True, 
                                          max_length = MAX_LENGTH, pad_to_max_length = True, use_fast = True)

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [48]:
def Tokenizer(sentences):
    input_ids = []
    attention_masks = []

    for sent in tqdm(sentences):
        bert_inp = tokenizer.encode_plus(sent, add_special_tokens = True , max_length = MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    return input_ids, attention_masks

In [49]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 7301.54it/s]


In [50]:
def create_model(config = MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape = (MAX_LENGTH,), name = 'input_token', dtype = 'int32')
    input_masks_in = tf.keras.layers.Input(shape =(MAX_LENGTH,), name = 'masked_token', dtype = 'int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation = 'relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)
    model = tf.keras.Model(inputs = [input_ids_in, input_masks_in], outputs = X)
    return model

In [51]:
model = create_model()

Downloading:   0%|          | 0.00/565M [00:00<?, ?B/s]

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [52]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model (TFXLNetModel)  TFXLNetModelOutput(  116718336   ['input_token[0][0]',            
                                last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [53]:
model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-6), metrics = ['accuracy'])

In [54]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath = './best_xlnet_model.hdf5', monitor = 'val_accuracy', verbos = 1, save_best_only = True, mode = 'max')
history = model.fit([input_ids_train, attention_masks_train],
                    y_train, 
                    batch_size = 32, 
                    epochs = 10, 
                    callbacks = [mc], 
                    validation_data = ([input_ids_valid, attention_masks_vaild], y_valid))

Epoch 1/10


105/105 [==============================] - 141s 1s/step - loss: 0.6067 - accuracy: 0.7136 - val_loss: 0.2859 - val_accuracy: 0.8655
Epoch 2/10
105/105 [==============================] - 125s 1s/step - loss: 0.3282 - accuracy: 0.8531 - val_loss: 0.1596 - val_accuracy: 0.8960
Epoch 3/10
105/105 [==============================] - 125s 1s/step - loss: 0.2264 - accuracy: 0.9025 - val_loss: 0.1272 - val_accuracy: 0.9713
Epoch 4/10
105/105 [==============================] - 126s 1s/step - loss: 0.1890 - accuracy: 0.9405 - val_loss: 0.0742 - val_accuracy: 0.9830
Epoch 5/10
105/105 [==============================] - 125s 1s/step - loss: 0.1598 - accuracy: 0.9458 - val_loss: 0.0754 - val_accuracy: 0.9848
Epoch 6/10
105/105 [==============================] - 126s 1s/step - loss: 0.1315 - accuracy: 0.9647 - val_loss: 0.0508 - val_accuracy: 0.9865
Epoch 7/10
105/105 [==============================] - 120s 1s/step - loss: 0.1237 - accuracy: 0.9650 - val_loss: 0.0636 - val_accuracy: 0.9857
Epoch 8/10

In [55]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 12s 328ms/step - loss: 0.0189 - accuracy: 0.9946


[0.01892862096428871, 0.994618833065033]

3. Theo như bài tập số 2 ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10

In [56]:
data = pd.read_csv('/content/spam.csv', encoding = 'ISO-8859-1')

In [57]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [58]:
data 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [59]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [60]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [61]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['gt'], test_size = 0.2, random_state = 0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

In [62]:
sentences = data['text']
labels = data['gt']
len(sentences), len(labels)

(5572, 5572)

In [63]:
MAX_LENGTH = 128
NUM_CLASSES = len(data['gt'].unique())
MODEL_NAME = 'xlnet-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

In [64]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case = False, add_special_tokens = True, 
                                          max_length = MAX_LENGTH, pad_to_max_length = True, use_fast = True)

In [65]:
def Tokenizer(sentences):
    input_ids = []
    attention_masks = []

    for sent in tqdm(sentences):
        bert_inp = tokenizer.encode_plus(sent, add_special_tokens = True , max_length = MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    return input_ids, attention_masks

In [66]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 7296.05it/s]


In [67]:
def create_model(config = MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape = (MAX_LENGTH,), name = 'input_token', dtype = 'int32')
    input_masks_in = tf.keras.layers.Input(shape =(MAX_LENGTH,), name = 'masked_token', dtype = 'int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(50, activation = 'relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(30, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)
    model = tf.keras.Model(inputs = [input_ids_in, input_masks_in], outputs = X)
    return model

In [68]:
model = create_model()

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [69]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model_1 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_token[0][0]',            
 )                              last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [70]:
model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-6), metrics = ['accuracy'])

In [71]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath = './best_xlnet_2_model.hdf5', monitor = 'val_accuracy', verbos = 1, save_best_only = True, mode = 'max')
history = model.fit([input_ids_train, attention_masks_train],
                    y_train, 
                    batch_size = 32, 
                    epochs = 10, 
                    callbacks = [mc], 
                    validation_data = ([input_ids_valid, attention_masks_vaild], y_valid))

Epoch 1/10


105/105 [==============================] - 137s 1s/step - loss: 0.5380 - accuracy: 0.7690 - val_loss: 0.2362 - val_accuracy: 0.8825
Epoch 2/10
105/105 [==============================] - 123s 1s/step - loss: 0.3055 - accuracy: 0.8519 - val_loss: 0.1317 - val_accuracy: 0.9462
Epoch 3/10
105/105 [==============================] - 124s 1s/step - loss: 0.2082 - accuracy: 0.9084 - val_loss: 0.0670 - val_accuracy: 0.9812
Epoch 4/10
105/105 [==============================] - 124s 1s/step - loss: 0.1393 - accuracy: 0.9497 - val_loss: 0.0591 - val_accuracy: 0.9821
Epoch 5/10
105/105 [==============================] - 125s 1s/step - loss: 0.1286 - accuracy: 0.9572 - val_loss: 0.0492 - val_accuracy: 0.9848
Epoch 6/10
105/105 [==============================] - 125s 1s/step - loss: 0.1111 - accuracy: 0.9659 - val_loss: 0.0448 - val_accuracy: 0.9865
Epoch 7/10
105/105 [==============================] - 125s 1s/step - loss: 0.0958 - accuracy: 0.9689 - val_loss: 0.0512 - val_accuracy: 0.9874
Epoch 8/10

In [72]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 11s 323ms/step - loss: 0.0183 - accuracy: 0.9946


[0.018298503011465073, 0.994618833065033]